In [1]:
# autoreload
%reload_ext autoreload
%autoreload 2

# Recreating the previous meta-analysis

i.e. '[Understanding coralline algal responses to ocean acidification: Meta-analysis and synthesis](https://onlinelibrary.wiley.com/doi/full/10.1111/gcb.15899)' (Cornwall et al., 2022)

The R code used for the analysis (Meta-analysis Figure 5-6.Rmd), along with the two .csv datasets (juvenile_raw_data(in).csv, juvenile_raw_data(in).csv) were provided by Ben Harvey.

The following is a translation of the original R code into Python to compare and contrast with our new analysis.


In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence
import matplotlib.pyplot as plt

import pandas as pd

from calcification import config

# load data
dat = pd.read_csv(config.tmp_data_dir / "effect_sizes.csv")

mods_formula = ""
model = smf.mixedlm(
    f"{yi} ~ {mods_formula}", 
    data,
    groups=data["original_doi"], 
    re_formula="~1", 
    vc_formula={"ID": "0 + C(ID)"}
).fit(reml=True)

    
# Compute Cook's Distance
influence = OLSInfluence(model)
cooks_dist = influence.cooks_distance[0]

# Determine Cook's distance threshold
threshold_val = np.mean(cooks_dist) + 2 * np.std(cooks_dist)
print(f"Cook's distance threshold: {threshold_val:.2f}")

# Plot Cook's Distance
plt.figure(figsize=(8, 5))
plt.plot(cooks_dist, marker='o', linestyle='-', label="Cook's Distance")
plt.axhline(y=threshold_val, color='r', linestyle='--', label="Threshold")
plt.xlabel("ID")
plt.ylabel("Cook's Distance (log scale)")
plt.yscale("log")
plt.legend()
plt.show()


NameError: name 'yi' is not defined

In [ ]:
import pandas as pd
import numpy as np
from rpy2 import robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr

# Activate automatic conversion between pandas and R dataframes
pandas2ri.activate()

# Import necessary R packages
base = importr('base')
metafor = importr('metafor')
stats = importr('stats')

# Load your dataset (example)
# Replace this with your actual data loading code
your_dataset = pd.DataFrame({
    'Study': np.repeat([1, 2, 3, 4, 5], 3),
    'ID': range(1, 16),
    'n': np.random.randint(10, 100, 15),
    'hedges_g': np.random.normal(0.3, 0.5, 15),
    'variance': np.random.uniform(0.01, 0.1, 15),
    'pH_change': np.random.uniform(-0.5, 0.5, 15),
    'Temperature_change': np.random.uniform(-2, 2, 15),
    'Geographic_region': np.random.choice(['Polar', 'Temperate', 'Tropical'], 15),
    'Taxonomic_group': np.random.choice(['Fish', 'Coral', 'Algae', 'Mollusks'], 15),
    'Light_intensity': np.random.uniform(100, 1000, 15),
    'Experimental_duration': np.random.randint(7, 100, 15)
})

# Convert pandas dataframe to R dataframe
r_dataframe = pandas2ri.py2rpy(your_dataset)

# Define the random effects structure - this is the same for all models
# Study/ID indicates nested random effects (IDs nested within Studies)
random_structure = robjects.Formula('~ 1 | Study/ID')
random_list = robjects.ListVector({'random': random_structure})

# 1. Null model (baseline heterogeneity)
null_model = metafor.rma_mv(
    yi=robjects.vectors.FloatVector(r_dataframe.rx2('hedges_g')),  # Access column from R dataframe directly
    V=robjects.vectors.FloatVector(r_dataframe.rx2('variance')),    # Access column from R dataframe directly
    data=r_dataframe,
    method="REML",
    random=random_list
)

# 2. Main effects model
main_formula = robjects.Formula('~ pH_change + Temperature_change + Geographic_region + Taxonomic_group + Light_intensity + Experimental_duration')
main_model = metafor.rma_mv(
    yi=robjects.vectors.FloatVector(r_dataframe.rx2('hedges_g')),
    V=robjects.vectors.FloatVector(r_dataframe.rx2('variance')),
    mods=main_formula,
    data=r_dataframe,
    method="REML",
    random=random_list
)

# 3. Interactive model (testing synergistic effects)
interaction_formula = robjects.Formula('~ pH_change * Temperature_change + Geographic_region + Taxonomic_group + Light_intensity + Experimental_duration')
interaction_model = metafor.rma_mv(
    yi=robjects.vectors.FloatVector(r_dataframe.rx2('hedges_g')),
    V=robjects.vectors.FloatVector(r_dataframe.rx2('variance')),
    mods=interaction_formula,
    data=r_dataframe,
    method="REML",
    random=random_list
)

# Compare models using ANOVA
# model_comparison = metafor.anova_rma(null_model, main_model, interaction_model)

# Print results
print("Null Model:")
print(robjects.r('capture.output')(null_model))
print("\nMain Effects Model:")
print(robjects.r('capture.output')(main_model))
print("\nInteraction Model:")
print(robjects.r('capture.output')(interaction_model))
print("\nModel Comparison:")
print(robjects.r('capture.output')(model_comparison))


In [ ]:
# access null_model as a variable
null_model_py = {}

# Get the names of elements in the R ListVector
r_names = robjects.r('names')(null_model)
r_names_py = robjects.vectors.StrVector(r_names)

# Extract each element by name
for name in r_names_py:
    try:
        # Try to convert to Python object
        null_model_py[name] = null_model.rx2(name)
        print(null_model_py[name])
    except:
        # If it fails, print a warning
        print(f"Warning: Could not convert {name} to Python object")

null_model_py
# print("Extracted null model components:")
# for key in null_model_py:
#     print(f"- {key}: {type(null_model_py[key])}")